<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%9A%D0%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Конечные автоматы Мили и Мура
Вариант 5

In [2]:
import numpy as np
import pandas as pd
import random
import copy
from prettytable import PrettyTable

##Построение таблицы истинности:

###Задача 1. Мили

In [ ]:
x0 = [0, 1]

#2 входа - v
v = [0, 1]
#2 состояния - x1/x2
#4 выхода - y1/y2/y3/y4

#По поводу символов: +(сумма по модулю 22), ^(стрелка пирса), ну амперсанд и v(конъюнкция и дизъюнкция)
#|(штрих шеффера), '(отрицание)
#->(импликация вроде) и ~(эквивалентность)

In [ ]:
def t1_x1(x, v):
  rezult = 0
  #v1 + (x2 | x1 + v2 v (v2 + (x2 ^ x2)))
  return rezult

def t1_x2(x, v):
  rezult = 0
  #v1 -> v1 | (x1 ' | v1 + x1 ^ v2)
  return rezult

def t1_y1(x, v):
  rezult = 0
  #x1 v x2 + x2 | x1 ~ x1 ~ (x2 ' -> x1)
  return rezult

def t1_y2(x, v):
  rezult = 0
  #x1 ~ x2 ^ (x1 + x2 | (x2 ~ x2))
  return rezult

def t1_y3(x, v):
  rezult = 0
  #x2 & x2 | x2 + x2 ^ x2
  return rezult

def t1_y4(x, v):
  rezult = 0
  #x1 v x1 ~ x2 & (x2 ^ x1 ^ x2)
  return rezult

###Задача 2. Мура

In [ ]:
x0 = [0, 0]
#2 входа - v
v = [0, 1]
#2 состояния - x
#4 выхода - y

##СКНФ/СДНФ
